In [1]:
import os
import pandas as pd
import geopandas as gp
pd.set_option('display.max_columns', None)

#Step 0: Load in ward shapefile and ER as gdf/df
oct22 = gp.read_file("../WI_Statewide_WardsOct14_2022/Statewide_WardsOct14_2022.shp")


#sample ER
atg = pd.read_excel("./Ward by Ward Report_Attorney General_0.xlsx",sheet_name=1, header=10)
atg = atg.fillna(method='ffill', axis=0)
atg.rename( columns={'Unnamed: 0':'county', 'Unnamed: 1':'RU', 'Unnamed: 2':'tot_votes'}, inplace=True )
atg = atg[atg["RU"]!="County Totals:"]


## GDF Join Preparation

In [2]:
#Create common RU identifier and test join ER and Ward shp


#Step 1: Create RU ID in ward gdf and RU ER
atg["county"] = atg["county"].str.rstrip(" ")
atg["ward_range"] = atg["RU"].str.split(" ").str[-1]
atg["mcd_str"] = atg["RU"].str.split(" ").str[0:-2].apply(lambda x: " ".join(x)).str.upper()
atg["county_mcd"] = atg["county"]+";"+atg["mcd_str"]
atg["county_mcd_ward"] = atg["county"]+";"+atg["mcd_str"]+";"+atg["ward_range"]

oct22["CTV"] = oct22["LABEL"].str.split(" ").str[-2]
oct22["MCD_NAME"] = oct22["LABEL"].str.split(" ").str[0:-3].apply(lambda x: " ".join(x)).str.upper()
wards22=oct22.copy()
wards22["county_mcd_ward"] = wards22["CNTY_NAME"].str.upper()+";"+wards22["CTV"].apply(lambda x: {"C":"CITY OF ", "V":"VILLAGE OF ", "T":"TOWN OF "}.get(x))+wards22["MCD_NAME"].str.upper()+";"+wards22["WARD_FIPS"].str.slice(start=-3)


#Step 2: From ER RUs create ward:RU dictionary (Alternatively do this from shp? Using ER as this is the goal set of RUs as oppsed to SHP with IDs from different time period - also ward range not clear in ward shp, that is needed from ER)
ward_dict={}
for ru in atg["county_mcd_ward"]:
    county_str = atg.loc[atg["county_mcd_ward"]==ru, "county"].to_string(index=False)
    mcd_str =  atg.loc[atg["county_mcd_ward"]==ru, "mcd_str"].to_string(index=False).upper()
    #print("RU: ",ru)
    input_wards = ru.split(";")[-1]
    
    if "," in ru.split(";")[-1] and "-" not in ru.split(";")[-1]:
        for x in list(input_wards.split(",")):
            ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
            ward_dict[ward_id_str] = ru
            #print(ward_id_str, ":", ru)
    elif '1-2A' in ru.split(";")[-1]:
        ward_dict["WINNEBAGO;TOWN OF WOLF RIVER;001"] = ru
        ward_dict["WINNEBAGO;TOWN OF WOLF RIVER;002"] = ru
    elif "-" in ru.split(";")[-1] and "," not in ru.split(";")[-1]:
        i = input_wards.split("-")[0]
        j = input_wards.split("-")[1]
        for x in range(int(i), int(j)+1):
            ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
            ward_dict[ward_id_str] = ru
            #print(ward_id_str, ":", ru)
    elif "," in ru.split(";")[-1] and "-" in ru.split(";")[-1]:
        for item in list(input_wards.split(",")):
            #print("item: ", item)
            if "-" in item:
                i = item.split("-")[0]
                j = item.split("-")[1]
                #print("i, j:", i, j)
                for x in range(int(i), int(j)+1):
                    ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
                    ward_dict[ward_id_str] = ru
                    #print(ward_id_str, ":", ru)
            else:
                x = item
                ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
                ward_dict[ward_id_str] = ru
                #print(ward_id_str, ":", ru)
    else:
        x = input_wards
        ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
        ward_dict[ward_id_str] = ru
        #print(ward_id_str, ":", ru)

        
ward_dict['BAYFIELD;TOWN OF GRANDVIEW;001']= 'BAYFIELD;TOWN OF GRAND VIEW;1'
ward_dict['CRAWFORD;VILLAGE OF MOUNT STERLING;001'] = 'CRAWFORD;VILLAGE OF MT. STERLING;1'
ward_dict['SAUK;VILLAGE OF LA VALLE;001'] = 'SAUK;VILLAGE OF LAVALLE;1'
ward_dict["VILAS;TOWN OF LAND O'LAKES;001"] = 'VILAS;TOWN OF LAND O-LAKES;1-2'
ward_dict["VILAS;TOWN OF LAND O'LAKES;002"] = 'VILAS;TOWN OF LAND O-LAKES;1-2'
ward_dict['WALWORTH;VILLAGE OF FONTANA-ON-GENEVA LAKE;001'] = 'WALWORTH;VILLAGE OF FONTANA;1-3'
ward_dict['WALWORTH;VILLAGE OF FONTANA-ON-GENEVA LAKE;002'] = 'WALWORTH;VILLAGE OF FONTANA;1-3'
ward_dict['WALWORTH;VILLAGE OF FONTANA-ON-GENEVA LAKE;003'] = 'WALWORTH;VILLAGE OF FONTANA;1-3'
ward_dict['WAUPACA;TOWN OF ST. LAWRENCE;001'] = 'WAUPACA;TOWN OF SAINT LAWRENCE;1-2'
ward_dict['WAUPACA;TOWN OF ST. LAWRENCE;002'] = 'WAUPACA;TOWN OF SAINT LAWRENCE;1-2'
ward_dict['WINNEBAGO;TOWN OF WOLF RIVER;02A'] = 'WINNEBAGO;TOWN OF WOLF RIVER;1-2A'
#test
ward_dict['CHIPPEWA;CITY OF CHIPPEWA FALLS;007'] = 'CHIPPEWA;CITY OF CHIPPEWA FALLS;8-7'
ward_dict['CHIPPEWA;CITY OF CHIPPEWA FALLS;008'] = 'CHIPPEWA;CITY OF CHIPPEWA FALLS;8-7'
ward_dict['KENOSHA;CITY OF KENOSHA;065'] = 'KENOSHA;TOWN OF SOMERS;2-3'



#Step 3: Apply ward:RU dictionary to ward gdf
wards22["RU_assignment"] = wards22["county_mcd_ward"].map(ward_dict)
gdf_wards_with_match = wards22[~wards22["RU_assignment"].isna()]


#Step 4: Check if any wards do not have valid RU assignment
gdf_wards_no_match = wards22[wards22["RU_assignment"].isna()]
print("Shape wards no match df: ", gdf_wards_no_match.shape)


#Step 5: Test join dissolved gdf<>df
ru_gdf = gdf_wards_with_match.dissolve(by="RU_assignment").reset_index().to_crs(wards22.crs)
prelim_shp_atger_gdf = ru_gdf.merge(atg, how="outer", left_on="RU_assignment", right_on="county_mcd_ward", indicator=True)
print("Wards not assigned: ",wards22["RU_assignment"].isna().shape,"\nWards with match shape: ", gdf_wards_with_match.shape, "\nFrom what should match, did not join: ", prelim_shp_atger_gdf[prelim_shp_atger_gdf["_merge"]!="both"].shape)


#All of this code in theory should create matches. 
#Main question is 1) how many IDs will not match and 2) are there other problems that will pop up
#Excluding those scenarios can move onto handle ER first - join ER together so that we can do one join for all ER... Unless bounds turn out to be different for some reason
#After that just checking and exporting

Shape wards no match df:  (1, 16)
Wards not assigned:  (6971,) 
Wards with match shape:  (6970, 16) 
From what should match, did not join:  (1, 27)


In [3]:
prelim_shp_atger_gdf[prelim_shp_atger_gdf["_merge"]=="right_only"][["county_mcd_ward_x","county_mcd_ward_y", "geometry"]]
#No population, exclude with note - reference email

,county_mcd_ward_x,county_mcd_ward_y,geometry
3626,NaN,EAU CLAIRE;CITY OF EAU CLAIRE;73,None


In [4]:
prelim_shp_atger_gdf[prelim_shp_atger_gdf["_merge"]=="left_only"][["county_mcd_ward_x","county_mcd_ward_y", "geometry"]]

,county_mcd_ward_x,county_mcd_ward_y,geometry


In [5]:
atg[atg["county_mcd_ward"].str.contains("EAU CLAIRE;CITY OF EAU CLAIRE;73")]

,county,RU,tot_votes,Josh Kaul,Eric Toney,SCATTERING,ward_range,mcd_str,county_mcd,county_mcd_ward
954,EAU CLAIRE,City of EAU CLAIRE Ward 73,0,0,0,0,73,CITY OF EAU CLAIRE,EAU CLAIRE;CITY OF EAU CLAIRE,EAU CLAIRE;CITY OF EAU CLAIRE;73


In [6]:
set_dict_notin_ogid = set(ward_dict.keys())-set(wards22["county_mcd_ward"])
set_in_ogid_notin_dict = set(wards22["county_mcd_ward"])-set(ward_dict.keys())

In [7]:
wards22.shape

(6971, 16)

In [8]:
len(ward_dict)

6983

In [9]:
len(set_in_ogid_notin_dict)

1

In [10]:
set_in_ogid_notin_dict

{'CALUMET;TOWN OF HARRISON;001'}


kenosha65 = oct22[oct22["LABEL"].str.upper().str.contains("KENOSHA - C 0065")]
july22 = gp.read_file("./Wisconsin_Redistricting_Wards_(2022)/Wisconsin_Redistricting_Wards_(2022).shp").to_crs(kenosha65.crs)
july22.to_file("./shp_to_check/july22.shp")
kenosha65.to_file("./shp_to_check/kenosha.shp")
oct22[(oct22["LABEL"].str.upper().str.contains("HARRISON - T"))&oct22["CNTY_NAME"].str.upper().str.contains("CALUMET")].to_file("./shp_to_check/harrison.shp")

In [11]:
#Step 6: Modify ER df column names

#Step 7: Join gdf<>df

#Step 8: Check results

#Step 9: Complete above steps for rest of ER

## Read in Election Results

In [15]:
#Step 0: Load in election results by election
from wi_2022_pber_column_dict import column_dict


def xlsx_format(df_path, sheet_name=1, header=10):
    df = pd.read_excel(df_path, sheet_name=sheet_name, header=header)
    df = df.fillna(method='ffill', axis=0)
    df.rename( columns={'Unnamed: 0':'county', 'Unnamed: 1':'RU', 'Unnamed: 2':'tot_votes'}, inplace=True )
    df = df[df["RU"]!="County Totals:"]

    df["county"] = df["county"].str.rstrip(" ")
    df["ward_range"] = df["RU"].str.split(" ").str[-1]
    df["mcd_str"] = df["RU"].str.split(" ").str[0:-2].apply(lambda x: " ".join(x)).str.upper()
    df["county_mcd"] = df["county"]+";"+df["mcd_str"]
    df["county_mcd_ward"] = df["county"]+";"+df["mcd_str"]+";"+df["ward_range"]
    return df


def get_candidate_party_dict(contest, df_path, sheet_name, header):
    df = pd.read_excel(df_path, sheet_name=sheet_name, header=[header-1, header])
    district = sheet_name 
    party_list = ["DEM", "REP", "IND"]
    no_match_list = []
    candidate_code_dict = {}
    for party in party_list: 
        try:
            print(party)
            candidate_code_dict[df.xs((party), axis=1).columns[0]] = contest+ str(district).zfill(2) + party[0:1]+ df.xs((party), axis=1).columns[0].split(" ")[-1][:3].upper()

        except:
            print(party, "hit except")
            no_match_list.append(str(district) + party)
            
    return candidate_code_dict, no_match_list
    

def read_in_df_and_col_dict(contest, path, dist_count):
    bigdf = pd.DataFrame()
    col_dict = {}
    no_match_list = []
    for i in range(1, dist_count):
        #i = i+1
        print(i)
        if i == 1:
            header=10
        else:
            header=9
        
        if contest!="GSU":
            new_return = get_candidate_party_dict(contest, path, i, header)
            print(new_return)
            new_dict = new_return[0]
            col_dict.update(new_dict)
            no_match_list.append(new_return[1])

        df = xlsx_format(path, i,header)
        bigdf = pd.concat([bigdf, df], ignore_index=True)
        #bigdf = bigdf.rename(columns=col_dict)
        
    return bigdf, col_dict, no_match_list


#Read in: atg, sos, tre, gov, uss
atg = xlsx_format("./Ward by Ward Report_Attorney General_0.xlsx")
atg["G22ATGOWRI"] = atg["SCATTERING"]
sos = xlsx_format("./Ward by Ward Report_Secretary of State_0.xlsx")
sos["G22SOSOWRI"] = sos["SCATTERING"]
tre = xlsx_format("./Ward by Ward Report_State Treasurer_0.xlsx")
tre["G22TREOWRI"] = tre["SCATTERING"]
gov = xlsx_format("./Ward by Ward Report_Governor_0.xlsx")
gov["G22GOVOWRI"] = gov["SCATTERING"]
uss = xlsx_format("./Ward by Ward Report_US Senate_0.xlsx")
uss["G22USSOWRI"] = uss["SCATTERING"]


#TODO readin: cong, sldu, sldl
#TODO question: will the RU ward combos/boundaries be different for this case?


#Step 1: Join election result dfs together (concat)
#erdf = #pd.concat([atg, sos, tre, gov, uss], axis=1)
erdf_1 = atg.merge(sos, how="outer", on=["county","RU","county_mcd_ward"], suffixes=('_atg', '_sos'))
erdf_2 = erdf_1.merge(tre, how="outer", on=["county","RU","county_mcd_ward"], suffixes=('_erdf1', '_tre'))
erdf_3 = erdf_2.merge(gov, how="outer", on=["county","RU","county_mcd_ward"], suffixes=('_erdf2', '_gov'))
erdf = erdf_3.merge(uss, how="outer", on=["county","RU","county_mcd_ward"], suffixes=('_erdf3', '_uss'))


#Step 2: Check if any RUs mismatch across dfs by size **note statewide might be different from state leg/cong?**
print("atg shape: ", atg.shape, "\nsos shape: ", sos.shape, "\ntre shape: ", tre.shape, "\ngov.shape: ", gov.shape, "\nuss.shape: ", uss.shape, "\nerdf shape: ", erdf.shape)


#Step 3: Translate column names from og to RDH format
erdf.columns = erdf.columns.str.replace(" ", "").str.replace("\n", "")
erdf = erdf.rename(columns=column_dict)
print("erdf columns: ", erdf.columns)
print(erdf.head(1))

#Step 4: Compare vote totals to original DF's

atg shape:  (3627, 11) 
sos shape:  (3627, 14) 
tre shape:  (3627, 12) 
gov.shape:  (3627, 14) 
uss.shape:  (3627, 12) 
erdf shape:  (3627, 52)
erdf columns:  Index(['county', 'RU', 'tot_votes_x', 'G22ATGDKAU', 'G22ATGRTON',
       'SCATTERING_x', 'ward_range_x', 'mcd_str_x', 'county_mcd_x',
       'county_mcd_ward', 'G22ATGOWRI', 'tot_votes_y', 'G22SOSDFOL',
       'G22SOSRLOU', 'G22SOSLHAR', 'G22SOSIMCF', 'Unnamed:7_n', 'SCATTERING_y',
       'ward_range_y', 'mcd_str_y', 'county_mcd_y', 'G22SOSOWRI',
       'tot_votes_n', 'G22TREDRIC', 'G22TRERLEI', 'G22TRECZUE', 'SCATTERING_n',
       'ward_range_n', 'mcd_str_n', 'county_mcd_n', 'G22TREOWRI',
       'tot_votes_m', 'G22GOVDEVE', 'G22GOVRMIC', 'G22GOVIBEG', 'G22GOVOWRI',
       'Unnamed:7_m', 'SCATTERING_m', 'ward_range_m', 'mcd_str_m',
       'county_mcd_m', 'G22GOVOWRI', '_merge', 'tot_votes', 'G22USSDBAR',
       'G22USSRJOH', 'G22USSIPAU', 'SCATTERING', 'ward_range', 'mcd_str',
       'county_mcd', 'G22USSOWRI'],
      dtype='obje

In [14]:
prelim_shp_atger_gdf[prelim_shp_atger_gdf["_merge"]=="right_only"][["county_mcd_ward_x","county_mcd_ward_y"]]

,county_mcd_ward_x,county_mcd_ward_y
3626,NaN,EAU CLAIRE;CITY OF EAU CLAIRE;73


In [15]:
prelim_shp_atger_gdf[prelim_shp_atger_gdf["_merge"]=="left_only"][["county_mcd_ward_x","county_mcd_ward_y"]]

,county_mcd_ward_x,county_mcd_ward_y


# Remaining contests read in

In [19]:



su = read_in_df_and_col_dict("GSU", "Ward by Ward Report_State Senator_0.xlsx", 18)
sl = read_in_df_and_col_dict("GSL", "./Ward by Ward Report_Representative to the Assembly_0.xlsx", 100)
cong = read_in_df_and_col_dict("GCON", "./Ward by Ward Report_Representative in Congress_0.xlsx", 9)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
1
DEM
REP
IND
IND hit except
({'Roberta Thelen': 'GSL01DTHE', 'Joel Kitchens': 'GSL01RKIT'}, ['1IND'])
2
DEM
REP
IND
IND hit except
({'Renee Gasch': 'GSL02DGAS', 'Shae  Sortwell': 'GSL02RSOR'}, ['2IND'])
3
DEM
DEM hit except
REP
IND
IND hit except
({'Ron  Tusler': 'GSL03RTUS'}, ['3DEM', '3IND'])
4
DEM
REP
IND
IND hit except
({'Derek  Teague': 'GSL04DTEA', 'David  Steffen': 'GSL04RSTE'}, ['4IND'])
5
DEM
REP
IND
IND hit except
({'Joseph Van Deurzen': 'GSL05DDEU', 'Joy Goeben': 'GSL05RGOE'}, ['5IND'])
6
DEM
REP
IND
IND hit except
({'William J. Switalla': 'GSL06DSWI', 'Peter  Schmidt': 'GSL06RSCH'}, ['6IND'])
7
DEM
REP
IND
IND hit except
({'Daniel Graan Riemer': 'GSL07DRIE', 'Zachary  Marshall': 'GSL07RMAR'}, ['7IND'])
8
DEM
REP
REP hit except
IND
IND hit except
({'Sylvia  Ortiz-Velez': 'GSL08DORT'}, ['8REP', '8IND'])
9
DEM
REP
IND
IND hit except
({'Marisabel  Cabrera': 'GSL09DCAB', 'Ryan Michael Antczak': 'GSL09RANT'}, ['9IND'])
10
DEM
REP
REP hit

In [20]:
set(su[0]["county_mcd_ward"])-set(sl[0]["county_mcd_ward"])

set()

In [21]:
set(sl[0]["county_mcd_ward"])-set(atg["county_mcd_ward"])

set()

In [38]:
su[0].shape

(1926, 38)

In [39]:
cong[0]["RU"].nunique()

3538

In [41]:
cong[0].shape

(3627, 31)

## DF <> GDF Join

In [25]:
erdf.columns

Index(['county', 'RU', 'tot_votes_x', 'G22ATGDKAU', 'G22ATGRTON',
       'SCATTERING_x', 'ward_range_x', 'mcd_str_x', 'county_mcd_x',
       'county_mcd_ward', 'G22ATGOWRI', 'tot_votes_y', 'G22SOSDFOL',
       'G22SOSRLOU', 'G22SOSLHAR', 'G22SOSIMCF', 'Unnamed:7_n', 'SCATTERING_y',
       'ward_range_y', 'mcd_str_y', 'county_mcd_y', 'G22SOSOWRI',
       'tot_votes_n', 'G22TREDRIC', 'G22TRERLEI', 'G22TRECZUE', 'SCATTERING_n',
       'ward_range_n', 'mcd_str_n', 'county_mcd_n', 'G22TREOWRI',
       'tot_votes_m', 'G22GOVDEVE', 'G22GOVRMIC', 'G22GOVIBEG', 'G22GOVOWRI',
       'Unnamed:7_m', 'SCATTERING_m', 'ward_range_m', 'mcd_str_m',
       'county_mcd_m', 'G22GOVOWRI', '_merge', 'tot_votes', 'G22USSDBAR',
       'G22USSRJOH', 'G22USSIPAU', 'SCATTERING', 'ward_range', 'mcd_str',
       'county_mcd', 'G22USSOWRI'],
      dtype='object')

In [26]:
test_merge = pd.merge(erdf, sl[0], how="outer", on=["county","RU","county_mcd_ward"], suffixes=("_erdf", "_sl"))

In [28]:
test_merge

,county,RU,tot_votes_x,G22ATGDKAU,G22ATGRTON,SCATTERING_x,ward_range_x,mcd_str_x,county_mcd_x,county_mcd_ward,G22ATGOWRI,tot_votes_y,G22SOSDFOL,G22SOSRLOU,G22SOSLHAR,G22SOSIMCF,Unnamed:7_n,SCATTERING_y,ward_range_y,mcd_str_y,county_mcd_y,G22SOSOWRI,tot_votes_n,G22TREDRIC,G22TRERLEI,G22TRECZUE,SCATTERING_n,ward_range_n,mcd_str_n,county_mcd_n,G22TREOWRI,tot_votes_m,G22GOVDEVE,G22GOVRMIC,G22GOVIBEG,G22GOVOWRI,Unnamed:7_m,SCATTERING_m,ward_range_m,mcd_str_m,county_mcd_m,G22GOVOWRI,_merge,tot_voteserdf,G22USSDBAR,G22USSRJOH,G22USSIPAU,SCATTERINGerdf,ward_rangeerdf,mcd_strerdf,county_mcderdf,G22USSOWRI,tot_votessl,Roberta Thelen,Joel Kitchens,SCATTERINGsl,ward_rangesl,mcd_strsl,county_mcdsl,Renee Gasch,Shae Sortwell,Ron Tusler,Derek Teague,David Steffen,Joseph Van Deurzen,Joy Goeben,William J. Switalla,Peter Schmidt,Dean Neubert (write-in),Daniel Graan Riemer,Zachary Marshall,Sylvia Ortiz-Velez,Marisabel Cabrera,Ryan Michael Antczak,Darrin Madison,Dora Drake,LaKeshia N. Myers,Greg Canady,Sarah Harrison,Tom Michalski,Robyn Vining,Keva Turner,Dave Maxey,Kalan Haywood,Supreme Moore Omokunde,Abie Eisenbach,Evan Goyke,Ryan Clancy,Christine M. Sinicki,Scott Hermann,Nathan M Jurowski,Jessie Rodriguez,Matt Brown,Janel Brandtjen,Deb Andraca,Purnima Nath,Bob Tatterson,Dan Knodl,Paul Tittl,Lisa Salgado,Terry A. Katsma,Amy Binsfeld,Chet Gerlach,Patty Schachtner,Gae Magnafici,Danielle Johnson,Clint Moses,Sarah Yacoub,Shannon Zimmerman,Brienne Brown,Ellen Schutt,Adam Jaramillo,Tyler August,Don Vruwink,Scott Johnson,Eileen Daniel,Rob Swearingen,Calvin Callahan,Todd Frederick,Ben Murray,Jeffrey Mursau,Maureen McCarville,William Penterman,Barbara Dittrich,Mark L Born,Steve Rydzewski (write-in),Kevin Petersen,Henry Fries,Alex Dallman,Theresa Valencia,Jon Plumer,Jenna Jacobson,Marisa Voelkel,Sue Conley,Spencer Zimmerman,Clinton Anderson,Jeff Klett,Melissa Ratcliff,Andrew McKinney,Jimmy Anderson,Lamonte Newsom,Samba Baldeh,Lynne Parrott,Travis Tranel,Michael Leuth,Tony Kurtz,Leah Spicer,Todd Novak,Joe Lavrenz,Jerry L. O'Connor,Michael Schraa,Lori Palmeri,Donnie Herman,Stefanie A. Holt,Nate Gustafson,Patrick Hayden,Dave Murphy,Lee Snodgrass,Andrew Fox,Mary Ann Rzeszutek,Rick Gundrum,Ty Bodden,Daniel E. Larsen,Robert Brooks,Max Winkels,Amanda Nedweski,Anthony Hammes,Robert Wittke,Robin Vos,Adam Steen (write-in),Joel Jacobsen (write-in),Tip McGuire,Ed Hibsch,Tod Ohnstad,Frank Petrick,Greta Neubauer,Carl Hutton,Jason D. Bennett,Rob Summerfield,Nate Otto,Karen Hurd,Lisa Boero,Donna M. Rozar,Remy Gomez,Nancy Lynn VanderMeer,Katrina Shankland,Scott Soik,Criste Greening,Scott S. Krug,Laura R. Gapske,Angie Sapik,John Adams,Chanz Green,David Armstrong,Francesca Hong,Shelia Stubbs,Lisa B. Subeck,Matt Neuhaus,Alex Joers,Victoria Fueger,Mike Bare,Jacob D. Luginbuhl,David Considine,Shellie Benish,Deborah Davis,Chuck Wichgers,Zachary Roper (write-in),Nik Rettinger,Chaz Self,Lu Ann Bird,Bob Donovan,Kristin Conway,Patrick Snyder,John Spiros,Elizabeth Riley,James W. Edming,Hannah Beauchamp-Pope,John Macco,Jane Benson,Elijah Behnke,Kristina Shelton,Micah J. Behnke,Jodi Emerson,Josh Stanley,Maria Bamonti,Treig E. Pronschinske,Alison H. Page,Warren L. Petryk,Steve Doyle,Ryan Huebsch,Jill Billings,Chris Woodard,Jayne M. Swiggum,Loren Oldenburg,Scott Allen,Christina Barry,Adam Neylon,Alec Thomas Dahms,Cindi Duchow
0,ADAMS,Town of ADAMS Wards 1-3,603,237,365,1,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,ADAMS;TOWN OF ADAMS;1-3,1,606,224,359,11,10,NaN,2,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,2,603,225,354,24,0,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,0,612,246,352,13,0,NaN,1,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,1,both,613,243,367,0,3,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,3,500,NaN,NaN,21,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [29]:
erdf.shape

(3627, 52)

In [30]:
test_merge["county_mcd_ward"].nunique()

3627

In [35]:
test_merge.merge(ru_gdf, how="outer", left_on="county_mcd_ward", right_on="RU_assignment", suffixes=("_test_merge", "_ru_gdf"))

,county,RU,tot_votes_x,G22ATGDKAU,G22ATGRTON,SCATTERING_x,ward_range_x,mcd_str_x,county_mcd_x,county_mcd_ward_test_merge,G22ATGOWRI,tot_votes_y,G22SOSDFOL,G22SOSRLOU,G22SOSLHAR,G22SOSIMCF,Unnamed:7_n,SCATTERING_y,ward_range_y,mcd_str_y,county_mcd_y,G22SOSOWRI,tot_votes_n,G22TREDRIC,G22TRERLEI,G22TRECZUE,SCATTERING_n,ward_range_n,mcd_str_n,county_mcd_n,G22TREOWRI,tot_votes_m,G22GOVDEVE,G22GOVRMIC,G22GOVIBEG,G22GOVOWRI,Unnamed:7_m,SCATTERING_m,ward_range_m,mcd_str_m,county_mcd_m,G22GOVOWRI,_merge,tot_voteserdf,G22USSDBAR,G22USSRJOH,G22USSIPAU,SCATTERINGerdf,ward_rangeerdf,mcd_strerdf,county_mcderdf,G22USSOWRI,tot_votessl,Roberta Thelen,Joel Kitchens,SCATTERINGsl,ward_rangesl,mcd_strsl,county_mcdsl,Renee Gasch,Shae Sortwell,Ron Tusler,Derek Teague,David Steffen,Joseph Van Deurzen,Joy Goeben,William J. Switalla,Peter Schmidt,Dean Neubert (write-in),Daniel Graan Riemer,Zachary Marshall,Sylvia Ortiz-Velez,Marisabel Cabrera,Ryan Michael Antczak,Darrin Madison,Dora Drake,LaKeshia N. Myers,Greg Canady,Sarah Harrison,Tom Michalski,Robyn Vining,Keva Turner,Dave Maxey,Kalan Haywood,Supreme Moore Omokunde,Abie Eisenbach,Evan Goyke,Ryan Clancy,Christine M. Sinicki,Scott Hermann,Nathan M Jurowski,Jessie Rodriguez,Matt Brown,Janel Brandtjen,Deb Andraca,Purnima Nath,Bob Tatterson,Dan Knodl,Paul Tittl,Lisa Salgado,Terry A. Katsma,Amy Binsfeld,Chet Gerlach,Patty Schachtner,Gae Magnafici,Danielle Johnson,Clint Moses,Sarah Yacoub,Shannon Zimmerman,Brienne Brown,Ellen Schutt,Adam Jaramillo,Tyler August,Don Vruwink,Scott Johnson,Eileen Daniel,Rob Swearingen,Calvin Callahan,Todd Frederick,Ben Murray,Jeffrey Mursau,Maureen McCarville,William Penterman,Barbara Dittrich,Mark L Born,Steve Rydzewski (write-in),Kevin Petersen,Henry Fries,Alex Dallman,Theresa Valencia,Jon Plumer,Jenna Jacobson,Marisa Voelkel,Sue Conley,Spencer Zimmerman,Clinton Anderson,Jeff Klett,Melissa Ratcliff,Andrew McKinney,Jimmy Anderson,Lamonte Newsom,Samba Baldeh,Lynne Parrott,Travis Tranel,Michael Leuth,Tony Kurtz,Leah Spicer,Todd Novak,Joe Lavrenz,Jerry L. O'Connor,Michael Schraa,Lori Palmeri,Donnie Herman,Stefanie A. Holt,Nate Gustafson,Patrick Hayden,Dave Murphy,Lee Snodgrass,Andrew Fox,Mary Ann Rzeszutek,Rick Gundrum,Ty Bodden,Daniel E. Larsen,Robert Brooks,Max Winkels,Amanda Nedweski,Anthony Hammes,Robert Wittke,Robin Vos,Adam Steen (write-in),Joel Jacobsen (write-in),Tip McGuire,Ed Hibsch,Tod Ohnstad,Frank Petrick,Greta Neubauer,Carl Hutton,Jason D. Bennett,Rob Summerfield,Nate Otto,Karen Hurd,Lisa Boero,Donna M. Rozar,Remy Gomez,Nancy Lynn VanderMeer,Katrina Shankland,Scott Soik,Criste Greening,Scott S. Krug,Laura R. Gapske,Angie Sapik,John Adams,Chanz Green,David Armstrong,Francesca Hong,Shelia Stubbs,Lisa B. Subeck,Matt Neuhaus,Alex Joers,Victoria Fueger,Mike Bare,Jacob D. Luginbuhl,David Considine,Shellie Benish,Deborah Davis,Chuck Wichgers,Zachary Roper (write-in),Nik Rettinger,Chaz Self,Lu Ann Bird,Bob Donovan,Kristin Conway,Patrick Snyder,John Spiros,Elizabeth Riley,James W. Edming,Hannah Beauchamp-Pope,John Macco,Jane Benson,Elijah Behnke,Kristina Shelton,Micah J. Behnke,Jodi Emerson,Josh Stanley,Maria Bamonti,Treig E. Pronschinske,Alison H. Page,Warren L. Petryk,Steve Doyle,Ryan Huebsch,Jill Billings,Chris Woodard,Jayne M. Swiggum,Loren Oldenburg,Scott Allen,Christina Barry,Adam Neylon,Alec Thomas Dahms,Cindi Duchow,RU_assignment,geometry,CNTY_FIPS,CNTY_NAME,COUSUBFP,WARD_FIPS,WARDID,SUPERID,ALDERID,LABEL,CON2021,ASM2021,SEN2021,CTV,MCD_NAME,county_mcd_ward_ru_gdf
0,ADAMS,Town of ADAMS Wards 1-3,603,237,365,1,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,ADAMS;TOWN OF ADAMS;1-3,1,606,224,359,11,10,NaN,2,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,2,603,225,354,24,0,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,0,612,246,352,13,0,NaN,1,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,1,both,613,243,367,0,3,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,3,500,NaN,NaN,21,1-3,TOWN OF ADAMS,ADAMS;TOWN OF ADAMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN